<a href="https://colab.research.google.com/github/fatemehm/Federated-Learning-IDS/blob/main/DNN_FL_based_IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet tensorflow-federated==0.20.0
#!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade nest-asyncio



     |████████████████████████████████| 819 kB 2.4 MB/s 
     |████████████████████████████████| 4.0 MB 13.7 MB/s 
     |████████████████████████████████| 65.1 MB 115 kB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 887 kB 38.1 MB/s 
     |████████████████████████████████| 251 kB 39.8 MB/s 
     |████████████████████████████████| 238 kB 55.5 MB/s 
     |████████████████████████████████| 497.9 MB 29 kB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 126 kB 69.7 MB/s 
     |████████████████████████████████| 121 kB 59.3 MB/s 
     |████████████████████████████████| 5.8 MB 48.5 MB/s 
     |████████████████████████████████| 1.4 MB 48.1 MB/s 
     |████████████████████████████████| 462 kB 69.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.2 requir

In [ ]:
#import tensorflow as tf
#print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.8.3


In [ ]:
#!python --version

Python 3.7.14


In [ ]:
# Mounts Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import numpy as np
import pandas as pd
import collections
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.layers import Dense
import matplotlib.pyplot as plt



Mounted at /content/drive


In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#import tensorflow_federated as tff 
#print(tff.federated_computation(lambda: 'Hello World'))


# **Wustl-ehms-2020 Dataset**

In [ ]:
df_wus = pd.read_csv('/content/drive/MyDrive/GlobeCom/clean_wustl.csv')

In [ ]:
df_wus.head()

,Unnamed: 0,Sport,SrcBytes,DstBytes,SrcLoad,DstLoad,SrcGap,DstGap,SIntPkt,DIntPkt,...,Rate,Temp,SpO2,Pulse_Rate,SYS,DIA,Heart_rate,Resp_Rate,ST,Label
0,0,58059.0,496.0,186.0,276914.0,92305.0,0.0,0.0,3.582333,1.9015,...,558.295,28.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,58062.0,496.0,186.0,230984.0,76995.0,0.0,0.0,4.294667,2.9015,...,465.694,28.9,0.0,0.0,0.0,0.0,78.0,17.0,0.4,0
2,2,58065.0,496.0,186.0,218470.0,72823.0,0.0,0.0,4.540667,3.2945,...,440.464,28.9,89.0,104.0,0.0,0.0,78.0,17.0,0.4,0
3,3,58067.0,496.0,186.0,203376.0,67792.0,0.0,0.0,4.877667,3.3320,...,410.032,28.9,89.0,104.0,0.0,0.0,79.0,17.0,0.4,0
4,4,58069.0,496.0,186.0,235723.0,78574.0,0.0,0.0,4.208333,2.8635,...,475.247,28.9,89.0,101.0,0.0,0.0,79.0,17.0,0.4,0


In [ ]:
import numpy as np
for i, df in enumerate(np.array_split(df_wus, 15)):
    df.to_csv('/content/drive/MyDrive/15-clients/'+f"wus{i+1}.csv", index=False)

In [ ]:
# Final features
feature_columns = ["Time", "Source", "Destination", "Protocol", "Length"]
df_treated = client1_ECU[feature_columns + ["Type"]]

In [ ]:
df_wus1 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus1.csv')

In [ ]:
df_wus1.head()

,Unnamed: 0,Sport,SrcBytes,DstBytes,SrcLoad,DstLoad,SrcGap,DstGap,SIntPkt,DIntPkt,...,Rate,Temp,SpO2,Pulse_Rate,SYS,DIA,Heart_rate,Resp_Rate,ST,Label
0,0,58059.0,496.0,186.0,276914.0,92305.0,0.0,0.0,3.582333,1.9015,...,558.295,28.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1,58062.0,496.0,186.0,230984.0,76995.0,0.0,0.0,4.294667,2.9015,...,465.694,28.9,0.0,0.0,0.0,0.0,78.0,17.0,0.4,0
2,2,58065.0,496.0,186.0,218470.0,72823.0,0.0,0.0,4.540667,3.2945,...,440.464,28.9,89.0,104.0,0.0,0.0,78.0,17.0,0.4,0
3,3,58067.0,496.0,186.0,203376.0,67792.0,0.0,0.0,4.877667,3.3320,...,410.032,28.9,89.0,104.0,0.0,0.0,79.0,17.0,0.4,0
4,4,58069.0,496.0,186.0,235723.0,78574.0,0.0,0.0,4.208333,2.8635,...,475.247,28.9,89.0,101.0,0.0,0.0,79.0,17.0,0.4,0


# ***Fifteen Clients***

In [ ]:
client1 = pd.read_csv('/content/drive/MyDrive/15-clients/wus1.csv')
client1_shuffled = client1.sample(frac=1, random_state = 13).reset_index(drop = True)

client2 = pd.read_csv('/content/drive/MyDrive/15-clients/wus2.csv')
client2_shuffled = client2.sample(frac=1, random_state = 13).reset_index(drop = True)

client3 = pd.read_csv('/content/drive/MyDrive/15-clients/wus3.csv')
client3_shuffled = client3.sample(frac=1, random_state = 13).reset_index(drop = True)

client4 = pd.read_csv('/content/drive/MyDrive/15-clients/wus4.csv')
client4_shuffled = client4.sample(frac=1, random_state = 13).reset_index(drop = True)

client5 = pd.read_csv('/content/drive/MyDrive/15-clients/wus5.csv')
client5_shuffled = client5.sample(frac=1, random_state = 13).reset_index(drop = True)

client6 = pd.read_csv('/content/drive/MyDrive/15-clients/wus6.csv')
client6_shuffled = client6.sample(frac=1, random_state = 13).reset_index(drop = True)

client7 = pd.read_csv('/content/drive/MyDrive/15-clients/wus7.csv')
client7_shuffled = client7.sample(frac=1, random_state = 13).reset_index(drop = True)

client8 = pd.read_csv('/content/drive/MyDrive/15-clients/wus8.csv')
client8_shuffled = client8.sample(frac=1, random_state = 13).reset_index(drop = True)

client9 = pd.read_csv('/content/drive/MyDrive/15-clients/wus9.csv')
client9_shuffled = client9.sample(frac=1, random_state = 13).reset_index(drop = True)

client10 = pd.read_csv('/content/drive/MyDrive/15-clients/wus10.csv')
client10_shuffled = client10.sample(frac=1, random_state = 13).reset_index(drop = True)

client11 = pd.read_csv('/content/drive/MyDrive/15-clients/wus11.csv')
client11_shuffled = client11.sample(frac=1, random_state = 13).reset_index(drop = True)

client12 = pd.read_csv('/content/drive/MyDrive/15-clients/wus12.csv')
client12_shuffled = client12.sample(frac=1, random_state = 13).reset_index(drop = True)

client13 = pd.read_csv('/content/drive/MyDrive/15-clients/wus13.csv')
client13_shuffled = client13.sample(frac=1, random_state = 13).reset_index(drop = True)

client14 = pd.read_csv('/content/drive/MyDrive/15-clients/wus14.csv')
client14_shuffled = client14.sample(frac=1, random_state = 13).reset_index(drop = True)

client15 = pd.read_csv('/content/drive/MyDrive/15-clients/wus15.csv')
client15_shuffled = client15.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
client1_shuffled.head()

,Unnamed: 0,Sport,SrcBytes,DstBytes,SrcLoad,DstLoad,SrcGap,DstGap,SIntPkt,DIntPkt,...,Rate,Temp,SpO2,Pulse_Rate,SYS,DIA,Heart_rate,Resp_Rate,ST,Label
0,248,58903.0,496.0,186.0,275632.0,91877.0,0.0,0.0,3.599000,2.3995,...,555.710,28.4,98.0,79.0,138.0,78.0,75.0,20.0,0.14,0
1,204,58751.0,496.0,186.0,293520.0,97840.0,0.0,0.0,3.379667,2.1535,...,591.774,28.9,97.0,76.0,138.0,78.0,70.0,12.0,0.16,0
2,142,58536.0,496.0,186.0,150829.0,50276.0,0.0,0.0,6.577000,2.8780,...,304.090,28.7,98.0,90.0,138.0,78.0,87.0,15.0,0.26,0
3,472,59656.0,496.0,186.0,178728.0,59576.0,0.0,0.0,5.550333,4.7425,...,360.339,26.8,98.0,98.0,138.0,78.0,97.0,20.0,0.30,1
4,370,59294.0,496.0,186.0,194535.0,64845.0,0.0,0.0,5.099333,3.0125,...,392.208,27.3,97.0,88.0,138.0,78.0,81.0,13.0,0.16,0


In [ ]:
client1_shuffled.Label.value_counts()

0    935
1    153
Name: Label, dtype: int64

In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
#BATCH_SIZE = 64 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['Label'] == 1]
    neg_df = dataset[dataset['Label'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Label')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED= 1337

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [client1_shuffled, client2_shuffled, client3_shuffled, client4_shuffled, client5_shuffled,
                    client6_shuffled, client7_shuffled, client8_shuffled, client9_shuffled, client10_shuffled,
                    client11_shuffled, client12_shuffled, client13_shuffled, client14_shuffled, client15_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Label'], axis=1))
    val_features = scaler.transform(val_df.drop(['Label'], axis=1))

    train_df[train_df.columns.difference(['Label'])] = train_features
    val_df[val_df.columns.difference(['Label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df,negative_ratio=2, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 35], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(35,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall(), AUC()]
        #tfa.metrics.F1Score(num_classes=2, threshold=0.5)
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[35,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[35,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[35,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

Run 100/100
Training time: 4.982948303222656e-05

Test time: 1.7881393432617188e-05

TIME: 478.51458191871643 seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
#100 epoch
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.906422),
                           ('precision', 0.70394737),
                           ('recall', 0.4976744),
                           ('loss', 0.3533172),
                           ('num_examples', 1635),
                           ('num_batches', 105)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.8343346), ('precision', 0.78944063), ('recall', 0.6859639), ('loss', 0.42554983), ('num_examples', 5493), ('num_batches', 15)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.5543419),
                            ('precision', 0.23150566),
                            ('recall', 0.1452758),
                            ('loss', 0.69515043),
                            ('num_examples', 5493),
                            ('num_batches', 15)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.75496083),
                            ('precision', 0.7320574),
                            ('recall', 0.41780448),
                            ('loss', 0.66365266),
                            ('num_examples', 5493),
                            ('num_batches', 15)]))]),
 OrderedDict([('broadcast', ()),
    

# **Ten Clients**

In [ ]:
client1 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus1.csv')
client1_shuffled = client1.sample(frac=1, random_state = 13).reset_index(drop = True)

client2 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus2.csv')
client2_shuffled = client2.sample(frac=1, random_state = 13).reset_index(drop = True)

client3 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus3.csv')
client3_shuffled = client3.sample(frac=1, random_state = 13).reset_index(drop = True)

client4 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus4.csv')
client4_shuffled = client4.sample(frac=1, random_state = 13).reset_index(drop = True)

client5 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus5.csv')
client5_shuffled = client5.sample(frac=1, random_state = 13).reset_index(drop = True)

client6 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus6.csv')
client6_shuffled = client6.sample(frac=1, random_state = 13).reset_index(drop = True)

client7 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus7.csv')
client7_shuffled = client7.sample(frac=1, random_state = 13).reset_index(drop = True)

client8 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus8.csv')
client8_shuffled = client8.sample(frac=1, random_state = 13).reset_index(drop = True)

client9 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus9.csv')
client9_shuffled = client9.sample(frac=1, random_state = 13).reset_index(drop = True)

client10 = pd.read_csv('/content/drive/MyDrive/Ten_clients/wus10.csv')
client10_shuffled = client10.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
#BATCH_SIZE = 64 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['Label'] == 1]
    neg_df = dataset[dataset['Label'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Label')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED= 1337

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [client1_shuffled, client2_shuffled, client3_shuffled, client4_shuffled, client5_shuffled,
                    client6_shuffled, client7_shuffled, client8_shuffled, client9_shuffled, client10_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Label'], axis=1))
    val_features = scaler.transform(val_df.drop(['Label'], axis=1))

    train_df[train_df.columns.difference(['Label'])] = train_features
    val_df[val_df.columns.difference(['Label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df,negative_ratio=2, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 35], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(35,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[35,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[35,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[35,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("\nTest time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {} seconds'.format(time_required))

Run 100/100
Training time: 6.771087646484375e-05

Test time: 2.1457672119140625e-05

TIME: 345.2668595314026 seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
#100 epoch
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9140244),
                           ('precision', 0.6505376),
                           ('recall', 0.61421317),
                           ('loss', 0.25559595),
                           ('num_examples', 1640),
                           ('num_batches', 110)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.8590229), ('precision', 0.7777199), ('recall', 0.8080043), ('loss', 0.3296115), ('num_examples', 5547), ('num_batches', 10)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.3369389),
                            ('precision', 0.30388162),
                            ('recall', 0.7663602),
                            ('loss', 0.73454934),
                            ('num_examples', 5547),
                            ('num_batches', 10)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.6666667),
                            ('precision', 0.0),
                            ('recall', 0.0),
                            ('loss', 0.93461937),
                            ('num_examples', 5547),
                            ('num_batches', 10)]))]),
 OrderedDict([('broadcast', ()),
              ('ag

# ***Five Clients***

In [ ]:
client1 = pd.read_csv('/content/drive/MyDrive/Five_clients/wus1.csv')
client1_shuffled = client1.sample(frac=1, random_state = 13).reset_index(drop = True)

client2 = pd.read_csv('/content/drive/MyDrive/Five_clients/wus2.csv')
client2_shuffled = client2.sample(frac=1, random_state = 13).reset_index(drop = True)

client3 = pd.read_csv('/content/drive/MyDrive/Five_clients/wus3.csv')
client3_shuffled = client3.sample(frac=1, random_state = 13).reset_index(drop = True)

client4 = pd.read_csv('/content/drive/MyDrive/Five_clients/wus4.csv')
client4_shuffled = client4.sample(frac=1, random_state = 13).reset_index(drop = True)

client5 = pd.read_csv('/content/drive/MyDrive/Five_clients/wus5.csv')
client5_shuffled = client5.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
#BATCH_SIZE = 64 #ECU
SEED=1337

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['Label'] == 1]
    neg_df = dataset[dataset['Label'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Label')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [client1_shuffled, client2_shuffled, client3_shuffled, client4_shuffled, client5_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Label'], axis=1))
    val_features = scaler.transform(val_df.drop(['Label'], axis=1))

    train_df[train_df.columns.difference(['Label'])] = train_features
    val_df[val_df.columns.difference(['Label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df,negative_ratio=2, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 35], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(35,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 5.602836608886719e-05
Test time: 2.2649765014648438e-05

TIME: 186.2287094593048seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
#100 epoch
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9070336),
                           ('precision', 0.564),
                           ('recall', 0.7663044),
                           ('loss', 0.35916635),
                           ('num_examples', 1635),
                           ('num_batches', 105)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.8972431), ('precision', 0.8810651), ('recall', 0.7996778), ('loss', 0.26238266), ('num_examples', 5586), ('num_batches', 9)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.57035446),
                            ('precision', 0.39740655),
                            ('recall', 0.55961335),
                            ('loss', 0.69042987),
                            ('num_examples', 5586),
                            ('num_batches', 9)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.7500895),
                            ('precision', 0.692562),
                            ('recall', 0.4500537),
                            ('loss', 0.6884337),
                            ('num_examples', 5586),
                            ('num_batches', 9)]))]),
 OrderedDict([('broadcast', ()),
        

# ***ECU-IoHT-Dataset***

In [ ]:
df_ecu = pd.read_csv('/content/drive/MyDrive/GlobeCom/clean_ECU.csv')

In [ ]:
df_ecu.head()

,Unnamed: 0,Time,Source,Destination,Protocol,Length,Type,Type of attack
0,0,0.0,67.0,69.0,0.0,0.0,0,0.0
1,1,1.0,64.0,68.0,0.0,0.0,0,0.0
2,2,2.0,67.0,69.0,0.0,0.0,0,0.0
3,3,3.0,45.0,44.0,2.0,23.0,1,3.0
4,4,4.0,64.0,68.0,0.0,0.0,0,0.0


In [ ]:
df_ecu.Type.value_counts()

0    87754
1    23453
Name: Type, dtype: int64

In [ ]:
import numpy as np
for i, df in enumerate(np.array_split(df_ecu, 5)):
    df.to_csv('/content/drive/MyDrive/5-ecu_clients/'+f"ecu{i+1}.csv", index=False)

# ***Fifteen Clients***

In [ ]:
client1_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu1.csv')
client1_ECU_shuffled = client1_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client2_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu2.csv')
client2_ECU_shuffled = client2_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client3_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu3.csv')
client3_ECU_shuffled = client3_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client4_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu4.csv')
client4_ECU_shuffled = client4_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client5_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu5.csv')
client5_ECU_shuffled = client5_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client6_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu6.csv')
client6_ECU_shuffled = client6_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client7_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu7.csv')
client7_ECU_shuffled = client7_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client8_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu8.csv')
client8_ECU_shuffled = client8_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client9_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu9.csv')
client9_ECU_shuffled = client9_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client10_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu10.csv')
client10_ECU_shuffled = client10_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client11_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu11.csv')
client11_ECU_shuffled = client11_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client12_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu12.csv')
client12_ECU_shuffled = client12_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client13_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu13.csv')
client13_ECU_shuffled = client13_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client14_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu14.csv')
client14_ECU_shuffled = client14_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client15_ECU = pd.read_csv('/content/drive/MyDrive/15-ecu-clients/ecu15.csv')
client15_ECU_shuffled = client15_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
client13_ECU_shuffled.Type.value_counts()

0    5587
1    1826
Name: Type, dtype: int64

In [ ]:
EPOCHS = 100
#BATCH_SIZE = 512
BATCH_SIZE = 1024 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['Type'] == 1]
    neg_df = dataset[dataset['Type'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Type')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
#########################################ECU#######################################################
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [client1_ECU_shuffled, client2_ECU_shuffled, client3_ECU_shuffled, client4_ECU_shuffled, client5_ECU_shuffled,
                    client6_ECU_shuffled, client7_ECU_shuffled, client8_ECU_shuffled, client9_ECU_shuffled, client10_ECU_shuffled,
                    client11_ECU_shuffled, client12_ECU_shuffled, client13_ECU_shuffled, client14_ECU_shuffled, client15_ECU_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.2, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    #encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Type'], axis=1))
    val_features = scaler.transform(val_df.drop(['Type'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['Type'])] = train_features
    val_df[val_df.columns.difference(['Type'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 6], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(6,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)


In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
 Training time: 8.273124694824219e-05
Test time: 3.9577484130859375e-05

TIME: 780.2444603443146seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9503259),
                           ('precision', 0.9397471),
                           ('recall', 0.81470335),
                           ('loss', 0.23011099),
                           ('num_examples', 22245),
                           ('num_batches', 1395)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.97579867), ('precision', 0.9417321), ('recall', 0.943886), ('loss', 0.0945784), ('num_examples', 88962), ('num_batches', 90)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.85826534),
                            ('precision', 0.62164074),
                            ('recall', 0.841551),
                            ('loss', 0.4128268),
                            ('num_examples', 88962),
                            ('num_batches', 90)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.80293834),
                            ('precision', 0.86918604),
                            ('recall', 0.079517044),
                            ('loss', 0.59458977),
                            ('num_examples', 88962),
                            ('num_batches', 90)]))]),
 OrderedDict([('broadcast', ()),
 

# ***Ten clients***

In [ ]:
client1_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu1.csv')
client1_ECU_shuffled = client1_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client2_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu2.csv')
client2_ECU_shuffled = client2_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client3_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu3.csv')
client3_ECU_shuffled = client3_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client4_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu4.csv')
client4_ECU_shuffled = client4_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client5_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu5.csv')
client5_ECU_shuffled = client5_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client6_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu6.csv')
client6_ECU_shuffled = client6_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client7_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu7.csv')
client7_ECU_shuffled = client7_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client8_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu8.csv')
client8_ECU_shuffled = client8_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client9_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu9.csv')
client9_ECU_shuffled = client9_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client10_ECU = pd.read_csv('/content/drive/MyDrive/10-ecu-clients/ecu10.csv')
client10_ECU_shuffled = client10_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
#BATCH_SIZE = 64 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['Type'] == 1]
    neg_df = dataset[dataset['Type'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Type')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED= 1337

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [client1_ECU_shuffled, client2_ECU_shuffled, client3_ECU_shuffled, client4_ECU_shuffled, client5_ECU_shuffled,
                    client6_ECU_shuffled, client7_ECU_shuffled, client8_ECU_shuffled, client9_ECU_shuffled, client10_ECU_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Type'], axis=1))
    val_features = scaler.transform(val_df.drop(['Type'], axis=1))

    train_df[train_df.columns.difference(['Type'])] = train_features
    val_df[val_df.columns.difference(['Type'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, negative_ratio=0, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 6], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(6,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 7.462501525878906e-05
Test time: 5.507469177246094e-05

TIME: 437.4289846420288seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.98472184),
                           ('precision', 0.93884164),
                           ('recall', 0.9918699),
                           ('loss', 0.116540805),
                           ('num_examples', 11127),
                           ('num_batches', 700)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.9884792), ('precision', 0.9487076), ('recall', 0.9994317), ('loss', 0.10921261), ('num_examples', 100080), ('num_batches', 100)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.8496003),
                            ('precision', 0.60116106),
                            ('recall', 0.8532866),
                            ('loss', 0.36474422),
                            ('num_examples', 100080),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.8129896),
                            ('precision', 0.59752923),
                            ('recall', 0.348172),
                            ('loss', 0.5830461),
                            ('num_examples', 100080),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
 

# ***Five clients***

In [ ]:
client1_ECU = pd.read_csv('/content/drive/MyDrive/5-ecu-clients/ecu1.csv')
client1_ECU_shuffled = client1_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client2_ECU = pd.read_csv('/content/drive/MyDrive/5-ecu-clients/ecu2.csv')
client2_ECU_shuffled = client2_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client3_ECU = pd.read_csv('/content/drive/MyDrive/5-ecu-clients/ecu3.csv')
client3_ECU_shuffled = client3_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client4_ECU = pd.read_csv('/content/drive/MyDrive/5-ecu-clients/ecu4.csv')
client4_ECU_shuffled = client4_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)

client5_ECU = pd.read_csv('/content/drive/MyDrive/5-ecu-clients/ecu5.csv')
client5_ECU_shuffled = client5_ECU.sample(frac=1, random_state = 13).reset_index(drop = True)



In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
#BATCH_SIZE = 64 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['Type'] == 1]
    neg_df = dataset[dataset['Type'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('Type')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [client1_ECU_shuffled, client2_ECU_shuffled, client3_ECU_shuffled, client4_ECU_shuffled, client5_ECU_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['Type'], axis=1))
    val_features = scaler.transform(val_df.drop(['Type'], axis=1))

    train_df[train_df.columns.difference(['Type'])] = train_features
    val_df[val_df.columns.difference(['Type'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df,negative_ratio=0, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 6], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(6,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )


In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[6,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 8.130073547363281e-05
Test time: 7.510185241699219e-05

TIME: 274.86662793159485seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9417528),
                           ('precision', 0.8264029),
                           ('recall', 0.90375274),
                           ('loss', 0.14649524),
                           ('num_examples', 11125),
                           ('num_batches', 700)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.97748846), ('precision', 0.91315734), ('recall', 0.98758733), ('loss', 0.13055034), ('num_examples', 100082), ('num_batches', 100)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.8283807),
                            ('precision', 0.55614644),
                            ('recall', 0.937795),
                            ('loss', 0.4665092),
                            ('num_examples', 100082),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.76147556),
                            ('precision', 0.42981905),
                            ('recall', 0.38790825),
                            ('loss', 0.45898172),
                            ('num_examples', 100082),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),

# ***ICU-Dataset***

In [ ]:
df_ICU1 = pd.read_csv('/content/drive/MyDrive/ICU/clean_ICU.csv')
df_ICU= df_ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
df_ICU.head()

,Unnamed: 0,frame.time_delta,frame.time_relative,frame.len,tcp.time_delta,tcp.len,tcp.ack,tcp.connection.fin,tcp.connection.rst,tcp.connection.sack,...,mqtt.msgtype,mqtt.qos,mqtt.retain,mqtt.topic_len,mqtt.ver,ip.proto,ip.ttl,label,tcp.flags.ack,tcp.flags.fin
0,106362,0.000002,124.501195,60.0,0.000387,0.0,30.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,128.0,1,1.0,1.0
1,42451,0.000001,4.103763,62.0,0.000001,8.0,29055.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,6.0,128.0,1,1.0,0.0
2,92698,0.000067,55.687269,1514.0,0.000067,1460.0,20765.0,0.0,0.0,0.0,...,3.0,1.0,0.0,65.0,0.0,6.0,64.0,1,1.0,0.0
3,87922,0.000082,51.196503,1514.0,0.000082,1460.0,25001.0,0.0,0.0,0.0,...,3.0,1.0,0.0,65.0,0.0,6.0,64.0,1,1.0,0.0
4,25956,0.000011,949.246403,72.0,0.000121,4.0,3872.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,6.0,64.0,0,1.0,0.0


In [ ]:
df_ICU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111882 entries, 0 to 111881
Data columns (total 31 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             111882 non-null  int64  
 1   frame.time_delta       111882 non-null  float64
 2   frame.time_relative    111882 non-null  float64
 3   frame.len              111882 non-null  float64
 4   tcp.time_delta         111882 non-null  float64
 5   tcp.len                111882 non-null  float64
 6   tcp.ack                111882 non-null  float64
 7   tcp.connection.fin     111882 non-null  float64
 8   tcp.connection.rst     111882 non-null  float64
 9   tcp.connection.sack    111882 non-null  float64
 10  tcp.connection.syn     111882 non-null  float64
 11  tcp.flags.push         111882 non-null  float64
 12  tcp.flags.reset        111882 non-null  float64
 13  tcp.flags.syn          111882 non-null  float64
 14  tcp.hdr_len            111882 non-nu

In [ ]:
df_ICU.loc[df_ICU['frame.time_delta'] == '0x00000012']

,Unnamed: 0,frame.time_delta,frame.time_relative,frame.len,tcp.flags,tcp.time_delta,tcp.len,tcp.ack,tcp.connection.fin,tcp.connection.rst,...,mqtt.qos,mqtt.retain,mqtt.topic_len,mqtt.ver,mqtt.willmsg_len,ip.proto,ip.ttl,label,tcp.flags.ack,tcp.flags.fin


In [ ]:
col = ['tcp.flags', 'tcp.checksum', 'mqtt.hdrflags', 'tcp.flags.ack', 'tcp.flags.fin']
df_ICU= df_ICU.drop(columns = col)

In [ ]:
df_ICU.head()

,Unnamed: 0,frame.time_delta,frame.time_relative,frame.len,tcp.time_delta,tcp.len,tcp.ack,tcp.connection.fin,tcp.connection.rst,tcp.connection.sack,...,mqtt.msgtype,mqtt.qos,mqtt.retain,mqtt.topic_len,mqtt.ver,ip.proto,ip.ttl,label,tcp.flags.ack,tcp.flags.fin
0,106362,0.000002,124.501195,60.0,0.000387,0.0,30.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,128.0,1,1.0,1.0
1,42451,0.000001,4.103763,62.0,0.000001,8.0,29055.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,6.0,128.0,1,1.0,0.0
2,92698,0.000067,55.687269,1514.0,0.000067,1460.0,20765.0,0.0,0.0,0.0,...,3.0,1.0,0.0,65.0,0.0,6.0,64.0,1,1.0,0.0
3,87922,0.000082,51.196503,1514.0,0.000082,1460.0,25001.0,0.0,0.0,0.0,...,3.0,1.0,0.0,65.0,0.0,6.0,64.0,1,1.0,0.0
4,25956,0.000011,949.246403,72.0,0.000121,4.0,3872.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,6.0,64.0,0,1.0,0.0


In [ ]:
import numpy as np
for i, df in enumerate(np.array_split(df_ICU, 5)):
    df.to_csv('/content/drive/MyDrive/5-ICU-Env/'+f"ICU-Env{i+1}.csv", index=False)

# ***Fifteen clients***

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU2 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env2.csv')
ICU2_shuffled = ICU2.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU3 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env3.csv')
ICU3_shuffled = ICU3.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU4 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env4.csv')
ICU4_shuffled = ICU4.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU5 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env5.csv')
ICU5_shuffled = ICU5.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU6 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env6.csv')
ICU6_shuffled = ICU6.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU7 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env7.csv')
ICU7_shuffled = ICU7.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU8 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env8.csv')
ICU8_shuffled = ICU8.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU9 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env9.csv')
ICU9_shuffled = ICU9.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU10 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env10.csv')
ICU10_shuffled = ICU10.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU11 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env11.csv')
ICU11_shuffled = ICU11.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU12 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env12.csv')
ICU12_shuffled = ICU12.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU13 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env13.csv')
ICU13_shuffled = ICU13.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU14 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env14.csv')
ICU14_shuffled = ICU14.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU15 = pd.read_csv('/content/drive/MyDrive/15-ICU-Env/ICU-Env15.csv')
ICU15_shuffled = ICU15.sample(frac=1, random_state = 13).reset_index(drop = True)





In [ ]:
ICU15_shuffled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7458 entries, 0 to 7457
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             7458 non-null   int64  
 1   frame.time_delta       7458 non-null   float64
 2   frame.time_relative    7458 non-null   float64
 3   frame.len              7458 non-null   float64
 4   tcp.time_delta         7458 non-null   float64
 5   tcp.len                7458 non-null   float64
 6   tcp.ack                7458 non-null   float64
 7   tcp.connection.fin     7458 non-null   float64
 8   tcp.connection.rst     7458 non-null   float64
 9   tcp.connection.sack    7458 non-null   float64
 10  tcp.connection.syn     7458 non-null   float64
 11  tcp.flags.push         7458 non-null   float64
 12  tcp.flags.reset        7458 non-null   float64
 13  tcp.flags.syn          7458 non-null   float64
 14  tcp.flags.urg          7458 non-null   float64
 15  tcp.

In [ ]:
ICU15_shuffled.label.value_counts()

1    5330
0    2128
Name: label, dtype: int64

In [ ]:
ex = pd.read_csv('/content/drive/MyDrive/10-ICU-Env1/ICU-Env1.csv')

In [ ]:
ex.head()

,Unnamed: 0,frame.time_delta,tcp.time_delta,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,mqtt.hdrflags,mqtt.msgtype,mqtt.qos,mqtt.retain,mqtt.ver,label
0,0,0.000000,0.000000,1,1,0,1,1.0,0.0,0.0,4.0,0
1,1,0.000053,0.000053,1,1,0,2,2.0,0.0,0.0,0.0,0
2,2,0.000043,0.000000,1,1,0,1,1.0,0.0,0.0,4.0,0
3,3,0.000020,0.000020,1,1,0,2,2.0,0.0,0.0,0.0,0
4,4,0.000016,0.000000,1,1,0,1,1.0,0.0,0.0,4.0,0


In [ ]:
EPOCHS = 100
#BATCH_SIZE = 512
BATCH_SIZE = 1024 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe1, batch_size=None):
    
    dataset = dataframe1.drop(['Unnamed: 0', 'frame.time_relative', 'frame.len', 'tcp.len', 'tcp.ack',
                               'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.sack', 
                               'tcp.connection.syn', 'tcp.flags.urg', 'tcp.hdr_len', 'tcp.pdu.size', 
                               'tcp.window_size_value', 'mqtt.clientid_len', 'mqtt.conack.val', 
                               'mqtt.conflag.passwd', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 
                               'mqtt.conflag.retain', 'mqtt.conflag.willflag', 'mqtt.dupflag', 
                               'mqtt.kalive', 'mqtt.len', 'mqtt.topic_len', 'mqtt.willmsg_len', 
                               'ip.proto', 'ip.ttl'], axis=1)
    count_ICU_0, count_ICU_1 = dataset.label.value_counts()

    df_ICU_0 = dataset[dataset['label'] == 0]
    df_ICU_1 = dataset[dataset['label'] == 1]

    df_ICU_0_under = df_ICU_0.sample(count_ICU_1)
    df_ICU_under = pd.concat([df_ICU_0_under, df_ICU_1], axis = 0)
    y = df_ICU_under.pop('label')

    dataset = tf.data.Dataset.from_tensor_slices((df_ICU_under.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
#########################################ECU#######################################################
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [ICU1_shuffled, ICU2_shuffled, ICU3_shuffled, ICU4_shuffled, ICU5_shuffled,
                    ICU6_shuffled, ICU7_shuffled, ICU8_shuffled, ICU9_shuffled, ICU10_shuffled,
                    ICU11_shuffled, ICU12_shuffled, ICU13_shuffled, ICU14_shuffled, ICU15_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.2, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = MinMaxScaler()
    #scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['label'], axis=1))
    val_features = scaler.transform(val_df.drop(['label'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['label'])] = train_features
    val_df[val_df.columns.difference(['label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 11], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(11,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 5.0067901611328125e-05
Test time: 2.0742416381835938e-05

TIME: 736.7280781269073seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.73210657),
                            ('precision', 0.7274721),
                            ('recall', 1.0),
                            ('loss', 0.49674082),
                            ('num_examples', 89502),
                            ('num_batches', 90)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.91754377),
                            ('precision', 0.89943427),
                            ('recall', 0.9960627),
                            ('loss', 0.27676818),
                            ('num_examples', 89502),
                            ('num_batches', 90)]))]),
 OrderedDict([('broadcast', ()),
        

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9905273),
                           ('precision', 0.9905643),
                           ('recall', 0.99634016),
                           ('loss', 0.047781102),
                           ('num_examples', 22380),
                           ('num_batches', 1410)]))])

# ***Ten Clients***

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU2 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env2.csv')
ICU2_shuffled = ICU2.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU3 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env3.csv')
ICU3_shuffled = ICU3.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU4 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env4.csv')
ICU4_shuffled = ICU4.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU5 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env5.csv')
ICU5_shuffled = ICU5.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU6 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env6.csv')
ICU6_shuffled = ICU6.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU7 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env7.csv')
ICU7_shuffled = ICU7.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU8 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env8.csv')
ICU8_shuffled = ICU8.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU9 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env9.csv')
ICU9_shuffled = ICU9.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU10 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env/ICU-Env10.csv')
ICU10_shuffled = ICU10.sample(frac=1, random_state = 13).reset_index(drop = True)


In [ ]:
ICU10_shuffled.label.value_counts()

1    8008
0    3180
Name: label, dtype: int64

In [ ]:
EPOCHS = 100
#BATCH_SIZE = 512
BATCH_SIZE = 1024 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe1, batch_size=None):
    
    dataset = dataframe1.drop(['Unnamed: 0', 'frame.time_relative', 'frame.len', 'tcp.len', 'tcp.ack',
                               'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.sack', 
                               'tcp.connection.syn', 'tcp.flags.urg', 'tcp.hdr_len', 'tcp.pdu.size', 
                               'tcp.window_size_value', 'mqtt.clientid_len', 'mqtt.conack.val', 
                               'mqtt.conflag.passwd', 'mqtt.conflag.qos', 'mqtt.conflag.reserved', 
                               'mqtt.conflag.retain', 'mqtt.conflag.willflag', 'mqtt.dupflag', 
                               'mqtt.kalive', 'mqtt.len', 'mqtt.topic_len', 'mqtt.willmsg_len', 
                               'ip.proto', 'ip.ttl'], axis=1)
    count_ICU_0, count_ICU_1 = dataset.label.value_counts()

    df_ICU_0 = dataset[dataset['label'] == 0]
    df_ICU_1 = dataset[dataset['label'] == 1]

    df_ICU_0_under = df_ICU_0.sample(count_ICU_1)
    df_ICU_under = pd.concat([df_ICU_0_under, df_ICU_1], axis = 0)
    y = df_ICU_under.pop('label')

    dataset = tf.data.Dataset.from_tensor_slices((df_ICU_under.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [ICU1_shuffled, ICU2_shuffled, ICU3_shuffled, ICU4_shuffled, ICU5_shuffled,
                    ICU6_shuffled, ICU7_shuffled, ICU8_shuffled, ICU9_shuffled, ICU10_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = MinMaxScaler()
    #scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['label'], axis=1))
    val_features = scaler.transform(val_df.drop(['label'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['label'])] = train_features
    val_df[val_df.columns.difference(['label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 11], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(11,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 0.00010204315185546875
Test time: 0.0002491474151611328

TIME: 520.1952741146088seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.7553728),
                            ('precision', 0.7912421),
                            ('recall', 0.89418703),
                            ('loss', 0.4480772),
                            ('num_examples', 100692),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.8538613),
                            ('precision', 0.8945812),
                            ('recall', 0.9021643),
                            ('loss', 0.3055107),
                            ('num_examples', 100692),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
  

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.98918676),
                           ('precision', 0.9947546),
                           ('recall', 0.990179),
                           ('loss', 0.032293595),
                           ('num_examples', 11190),
                           ('num_batches', 700)]))])

# ***Five clients***

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/5-ICU-Env/ICU-Env1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU2 = pd.read_csv('/content/drive/MyDrive/5-ICU-Env/ICU-Env2.csv')
ICU2_shuffled = ICU2.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU3 = pd.read_csv('/content/drive/MyDrive/5-ICU-Env/ICU-Env3.csv')
ICU3_shuffled = ICU3.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU4 = pd.read_csv('/content/drive/MyDrive/5-ICU-Env/ICU-Env4.csv')
ICU4_shuffled = ICU4.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU5 = pd.read_csv('/content/drive/MyDrive/5-ICU-Env/ICU-Env5.csv')
ICU5_shuffled = ICU5.sample(frac=1, random_state = 13).reset_index(drop = True)


In [ ]:
EPOCHS = 100
#BATCH_SIZE = 512
BATCH_SIZE = 1024 #ECU

In [ ]:
ICU5_shuffled.label.value_counts()

1    16081
0     6295
Name: label, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe1, batch_size=None):
    
    dataset = dataframe1.drop(['Unnamed: 0', 'frame.time_relative', 'frame.len', 'tcp.len', 'tcp.ack',
                               'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.sack', 
                               'tcp.connection.syn', 'tcp.hdr_len', 'tcp.pdu.size', 
                               'tcp.window_size_value', 'mqtt.clientid_len', 'mqtt.dupflag', 
                               'mqtt.kalive', 'mqtt.len', 'mqtt.topic_len', 
                               'ip.proto', 'ip.ttl'], axis=1)
    count_ICU_0, count_ICU_1 = dataset.label.value_counts()

    df_ICU_0 = dataset[dataset['label'] == 0]
    df_ICU_1 = dataset[dataset['label'] == 1]

    df_ICU_0_under = df_ICU_0.sample(count_ICU_1)
    df_ICU_under = pd.concat([df_ICU_0_under, df_ICU_1], axis = 0)
    y = df_ICU_under.pop('label')

    dataset = tf.data.Dataset.from_tensor_slices((df_ICU_under.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
#########################################ECU#######################################################
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [ICU1_shuffled, ICU2_shuffled, ICU3_shuffled, ICU4_shuffled, ICU5_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.1, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = MinMaxScaler() 
    train_features = scaler.fit_transform(train_df.drop(['label'], axis=1))
    val_features = scaler.transform(val_df.drop(['label'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['label'])] = train_features
    val_df[val_df.columns.difference(['label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df,batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 11], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(11,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[11,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

**test overfit**

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 9.894371032714844e-05
Test time: 6.461143493652344e-05

TIME: 288.8207836151123seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.78282285),
                            ('precision', 0.976583),
                            ('recall', 0.7136255),
                            ('loss', 0.6614121),
                            ('num_examples', 100692),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.81392765),
                            ('precision', 0.9756128),
                            ('recall', 0.75895137),
                            ('loss', 0.6597543),
                            ('num_examples', 100692),
                            ('num_batches', 100)]))]),
 OrderedDict([('broadcast', ()),
 

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.99955314),
                           ('precision', 0.9993781),
                           ('recall', 1.0),
                           ('loss', 0.0025791076),
                           ('num_examples', 22380),
                           ('num_batches', 1400)]))])

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 4.1961669921875e-05
Test time: 2.1457672119140625e-05

TIME: 218.78809237480164seconds


In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9996872),
                           ('precision', 0.9995646),
                           ('recall', 1.0),
                           ('loss', 0.004795919),
                           ('num_examples', 22380),
                           ('num_batches', 1400)]))])

In [ ]:
print(metrics)

OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('binary_accuracy', 0.99960893), ('precision', 0.9994539), ('recall', 1.0), ('loss', 0.0059961043), ('num_examples', 89502), ('num_batches', 90)]))])


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.66032046),
                            ('precision', 0.6990195),
                            ('recall', 0.9226446),
                            ('loss', 0.65274227),
                            ('num_examples', 89502),
                            ('num_batches', 90)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.71603984),
                            ('precision', 0.7159383),
                            ('recall', 1.0),
                            ('loss', 0.3738972),
                            ('num_examples', 89502),
                            ('num_batches', 90)]))]),
 OrderedDict([('broadcast', ()),
          

# ***ICU-Patient***

In [ ]:
df_ICU_PAI1 = pd.read_csv('/content/drive/MyDrive/ICU-Patient/clean_ICU-PAI.csv')
df_ICU_PAI= df_ICU_PAI1.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
import numpy as np
for i, df in enumerate(np.array_split(df_ICU_PAI, 5)):
    df.to_csv('/content/drive/MyDrive/5-ICU-PAI/'+f"ICU-Pai{i+1}.csv", index=False)

# ***Fifteenclients***

In [ ]:
EPOCHS = 100
#BATCH_SIZE = 512
BATCH_SIZE = 1024 #ECU

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/10-ICU-Env1/ICU-Env1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
ICU1.head()

,Unnamed: 0,frame.time_delta,tcp.time_delta,tcp.flags.ack,tcp.flags.push,tcp.flags.reset,mqtt.hdrflags,mqtt.msgtype,mqtt.qos,mqtt.retain,mqtt.ver,label
0,0,0.000000,0.000000,1,1,0,1,1.0,0.0,0.0,4.0,0
1,1,0.000053,0.000053,1,1,0,2,2.0,0.0,0.0,0.0,0
2,2,0.000043,0.000000,1,1,0,1,1.0,0.0,0.0,4.0,0
3,3,0.000020,0.000020,1,1,0,2,2.0,0.0,0.0,0.0,0
4,4,0.000016,0.000000,1,1,0,1,1.0,0.0,0.0,4.0,0


In [ ]:
ICU1_shuffled.label.value_counts()

0    18870
Name: label, dtype: int64

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU2 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai2.csv')
ICU2_shuffled = ICU2.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU3 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai3.csv')
ICU3_shuffled = ICU3.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU4 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai4.csv')
ICU4_shuffled = ICU4.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU5 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai5.csv')
ICU5_shuffled = ICU5.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU6 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai6.csv')
ICU6_shuffled = ICU6.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU7 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai7.csv')
ICU7_shuffled = ICU7.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU8 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai8.csv')
ICU8_shuffled = ICU8.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU9 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai9.csv')
ICU9_shuffled = ICU9.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU10 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai10.csv')
ICU10_shuffled = ICU10.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU11 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai11.csv')
ICU11_shuffled = ICU11.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU12 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai12.csv')
ICU12_shuffled = ICU12.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU13 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai13.csv')
ICU13_shuffled = ICU13.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU14 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai14.csv')
ICU14_shuffled = ICU14.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU15 = pd.read_csv('/content/drive/MyDrive/15-ICU-PAI/ICU-Pai15.csv')
ICU15_shuffled = ICU15.sample(frac=1, random_state = 13).reset_index(drop = True)

In [ ]:
ICU15_shuffled.head()

,Unnamed: 0,frame.time_delta,frame.time_relative,frame.len,tcp.time_delta,tcp.len,tcp.ack,tcp.connection.fin,tcp.connection.rst,tcp.connection.sack,...,mqtt.qos,mqtt.retain,mqtt.topic_len,mqtt.ver,mqtt.willmsg_len,ip.proto,ip.ttl,label,tcp.flags.ack,tcp.flags.fin
0,128105,0.000649,45.316170,62.0,0.000649,8.0,516748.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,128.0,1,1.0,0.0
1,115239,0.000001,24.268071,116.0,0.000001,62.0,4301.0,0.0,0.0,0.0,...,1.0,0.0,66.0,0.0,0.0,6.0,64.0,1,1.0,0.0
2,106958,0.000000,12.404392,54.0,0.000000,0.0,8433.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,64.0,1,1.0,0.0
3,71030,0.000159,5972.034364,80.0,1.997201,12.0,203.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,6.0,64.0,0,1.0,0.0
4,56388,0.003811,4341.044473,78.0,1.984205,10.0,149.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,6.0,64.0,0,1.0,0.0


In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['label'] == 1]
    neg_df = dataset[dataset['label'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('label')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [ICU1_shuffled, ICU2_shuffled, ICU3_shuffled, ICU4_shuffled, ICU5_shuffled,
                    ICU6_shuffled, ICU7_shuffled, ICU8_shuffled, ICU9_shuffled, ICU10_shuffled,
                    ICU11_shuffled, ICU12_shuffled, ICU13_shuffled, ICU14_shuffled, ICU15_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.2, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = MinMaxScaler()
    #scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['label'], axis=1))
    val_features = scaler.transform(val_df.drop(['label'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['label'])] = train_features
    val_df[val_df.columns.difference(['label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 37], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(37,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 8.20159912109375e-05
Test time: 5.245208740234375e-05

TIME: 729.119517326355seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.62552375),
                            ('precision', 0.59076357),
                            ('recall', 0.87104315),
                            ('loss', 0.60222965),
                            ('num_examples', 125538),
                            ('num_batches', 135)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.91961795),
                            ('precision', 0.8760618),
                            ('recall', 0.98168),
                            ('loss', 0.27520156),
                            ('num_examples', 125538),
                            ('num_batches', 135)]))]),
 OrderedDict([('broadcast', ()),

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.9995222),
                           ('precision', 0.9990594),
                           ('recall', 1.0),
                           ('loss', 0.0020721052),
                           ('num_examples', 31395),
                           ('num_batches', 1965)]))])

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 4.9591064453125e-05
Test time: 2.0265579223632812e-05

TIME: 511.685916185379seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.85037994),
                            ('precision', 0.7805969),
                            ('recall', 0.98395437),
                            ('loss', 0.38504314),
                            ('num_examples', 125538),
                            ('num_batches', 135)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.99083143),
                            ('precision', 0.9982297),
                            ('recall', 0.9838142),
                            ('loss', 0.055349015),
                            ('num_examples', 125538),
                            ('num_batches', 135)]))]),
 OrderedDict([('broadcast', ()

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.99939483),
                           ('precision', 0.99880886),
                           ('recall', 1.0),
                           ('loss', 0.009279552),
                           ('num_examples', 31395),
                           ('num_batches', 1965)]))])

# ***Ten clients***

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU2 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai2.csv')
ICU2_shuffled = ICU2.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU3 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai3.csv')
ICU3_shuffled = ICU3.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU4 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai4.csv')
ICU4_shuffled = ICU4.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU5 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai5.csv')
ICU5_shuffled = ICU5.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU6 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai6.csv')
ICU6_shuffled = ICU6.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU7 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai7.csv')
ICU7_shuffled = ICU7.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU8 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai8.csv')
ICU8_shuffled = ICU8.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU9 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai9.csv')
ICU9_shuffled = ICU9.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU10 = pd.read_csv('/content/drive/MyDrive/10-ICU-PAI/ICU-Pai10.csv')
ICU10_shuffled = ICU10.sample(frac=1, random_state = 13).reset_index(drop = True)



In [ ]:
EPOCHS = 100
#BATCH_SIZE = 512
BATCH_SIZE = 1024 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['label'] == 1]
    neg_df = dataset[dataset['label'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('label')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [ICU1_shuffled, ICU2_shuffled, ICU3_shuffled, ICU4_shuffled, ICU5_shuffled,
                    ICU6_shuffled, ICU7_shuffled, ICU8_shuffled, ICU9_shuffled, ICU10_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.2, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['label'], axis=1))
    val_features = scaler.transform(val_df.drop(['label'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['label'])] = train_features
    val_df[val_df.columns.difference(['label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 37], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(37,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 6.318092346191406e-05
Test time: 2.193450927734375e-05

TIME: 357.8170130252838seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.5275404),
                            ('precision', 0.5214659),
                            ('recall', 0.89655066),
                            ('loss', 0.5456056),
                            ('num_examples', 125543),
                            ('num_batches', 130)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.9739611),
                            ('precision', 0.95646554),
                            ('recall', 0.99420685),
                            ('loss', 0.25449586),
                            ('num_examples', 125543),
                            ('num_batches', 130)]))]),
 OrderedDict([('broadcast', ()),

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.99952215),
                           ('precision', 0.9990682),
                           ('recall', 1.0),
                           ('loss', 0.0017967706),
                           ('num_examples', 31390),
                           ('num_batches', 1970)]))])

# ***Five clients***

In [ ]:
ICU1 = pd.read_csv('/content/drive/MyDrive/5-ICU-PAI/ICU-Pai1.csv')
ICU1_shuffled = ICU1.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU2 = pd.read_csv('/content/drive/MyDrive/5-ICU-PAI/ICU-Pai2.csv')
ICU2_shuffled = ICU2.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU3 = pd.read_csv('/content/drive/MyDrive/5-ICU-PAI/ICU-Pai3.csv')
ICU3_shuffled = ICU3.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU4 = pd.read_csv('/content/drive/MyDrive/5-ICU-PAI/ICU-Pai4.csv')
ICU4_shuffled = ICU4.sample(frac=1, random_state = 13).reset_index(drop = True)

ICU5 = pd.read_csv('/content/drive/MyDrive/5-ICU-PAI/ICU-Pai5.csv')
ICU5_shuffled = ICU5.sample(frac=1, random_state = 13).reset_index(drop = True)



In [ ]:
ICU5_shuffled.head()

,Unnamed: 0,frame.time_delta,frame.time_relative,frame.len,tcp.time_delta,tcp.len,tcp.ack,tcp.connection.fin,tcp.connection.rst,tcp.connection.sack,...,mqtt.qos,mqtt.retain,mqtt.topic_len,mqtt.ver,mqtt.willmsg_len,ip.proto,ip.ttl,label,tcp.flags.ack,tcp.flags.fin
0,51380,0.000038,3791.030490,78.0,1.997875,10.0,131.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,6.0,64.0,0,1.0,0.0
1,78226,0.000257,0.142426,74.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,64.0,1,0.0,0.0
2,4606,0.000193,1006.960671,86.0,1.999598,18.0,37.0,0.0,0.0,0.0,...,0.0,0.0,12.0,0.0,0.0,6.0,64.0,0,1.0,0.0
3,119231,0.000033,34.018815,54.0,0.000033,0.0,14329.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,64.0,1,1.0,0.0
4,130438,0.000000,48.395294,62.0,0.000000,8.0,520832.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,128.0,1,1.0,0.0


In [ ]:
EPOCHS = 100
BATCH_SIZE = 1024
#BATCH_SIZE = 1024 #ECU

In [ ]:
from sklearn.preprocessing import LabelEncoder

def make_tf_dataset(dataframe, negative_ratio=None, batch_size=None):
  
    dataset = dataframe.drop(['Unnamed: 0'], axis=1)#, 'SrcGap', 'DstGap', 'DIntPktAct','sMinPktSz', 'Trans'
    # Class balancing
    pos_df = dataset[dataset['label'] == 1]
    neg_df = dataset[dataset['label'] == 0]
    if negative_ratio:
        neg_df = neg_df.iloc[random.sample(range(0, len(neg_df)), len(pos_df)*negative_ratio), :]
    balanced_df = pd.concat([pos_df, neg_df], ignore_index=True, sort=False)

    y = balanced_df.pop('label')

    dataset = tf.data.Dataset.from_tensor_slices((balanced_df.values, y.to_frame().values))
    dataset = dataset.shuffle(4048, seed=SEED) #2048
    if batch_size:
        dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
SEED=1337

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
train_data, val_data = [], []

for client_data in [ICU1_shuffled, ICU2_shuffled, ICU3_shuffled, ICU4_shuffled, ICU5_shuffled]:
                     
                   
    train_df, val_df = train_test_split(client_data, test_size=0.2, random_state=1337)

    # Scaling (Standardization actually hurts performance) 
    encoder = LabelEncoder()
    scaler = StandardScaler() 
    train_features = scaler.fit_transform(train_df.drop(['label'], axis=1))
    val_features = scaler.transform(val_df.drop(['label'], axis=1))
  
    #encoder = LabelEncoder()
    #y1 = encoder.fit_transform(y)
    #Y= pd.get_dummies(y1).values
    train_df[train_df.columns.difference(['label'])] = train_features
    val_df[val_df.columns.difference(['label'])] = val_features

    # TF Datasets
    train_data.append(make_tf_dataset(train_df, batch_size=BATCH_SIZE)) #negative_ratio=2
    val_data.append(make_tf_dataset(val_df, batch_size=16))

In [ ]:
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall, BinaryAccuracy
def input_spec():
    return (
        tf.TensorSpec([None, 37], tf.float64),
        tf.TensorSpec([None, 1], tf.int64)
    )

def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(37,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    return tff.learning.from_keras_model(
        model,
        input_spec=input_spec(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), Precision(), Recall()]
        )

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam(learning_rate=0.05)#learning_rate=0.01
)

In [ ]:
str(trainer.initialize.type_signature)

'( -> <model=<trainable=<float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,non_trainable=<>>,optimizer_state=<int64,float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1],float32[37,64],float32[64],float32[64,64],float32[64],float32[64,32],float32[32],float32[32,1],float32[1]>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [ ]:
state = trainer.initialize()

In [ ]:
import time
start = time.time()
start_time = time.time()
end = time.time()
diff=end-start
starttest = time.time()  
endtest =time.time()
difftest = endtest-starttest
#state = trainer.initialize()
train_hist = []
for i in range(EPOCHS):
    state, metrics = trainer.next(state, train_data)
    train_hist.append(metrics)

    print(f"\rRun {i+1}/{EPOCHS}", end="")
endtest =time.time()
#difftest = endtest-starttest
print("\nTraining time: " + str(diff))
print("Test time: " + str(difftest))
time_required = time.time() - start_time
print('\nTIME: {}seconds'.format(time_required))

Run 100/100
Training time: 5.7220458984375e-05
Test time: 2.09808349609375e-05

TIME: 227.73668146133423seconds


In [ ]:
train_hist

[OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.63597333),
                            ('precision', 0.588615),
                            ('recall', 0.95518386),
                            ('loss', 0.6674297),
                            ('num_examples', 125543),
                            ('num_batches', 125)]))]),
 OrderedDict([('broadcast', ()),
              ('aggregation',
               OrderedDict([('mean_value', ()), ('mean_weight', ())])),
              ('train',
               OrderedDict([('binary_accuracy', 0.9163713),
                            ('precision', 0.862259),
                            ('recall', 0.9953391),
                            ('loss', 0.26607782),
                            ('num_examples', 125543),
                            ('num_batches', 125)]))]),
 OrderedDict([('broadcast', ()),
  

In [ ]:
evaluator = tff.learning.build_federated_evaluation(model_fn)

In [ ]:
federated_metrics = evaluator(state.model, val_data)
federated_metrics

OrderedDict([('eval',
              OrderedDict([('binary_accuracy', 0.99958587),
                           ('precision', 0.9991868),
                           ('recall', 1.0),
                           ('loss', 0.0016059136),
                           ('num_examples', 31390),
                           ('num_batches', 1965)]))])